# Plot

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math as m
import mplstereonet
import pandas as pd
%matplotlib inline


In [ ]:
def theta_phi(filepath, sph):
    files = os.listdir(filepath)
    df = pd.DataFrame(sph, columns=['r', 'theta', 'phi'])
    theta = [m.degrees(df['theta'].iloc[i]) for i in range(len(df))]
    phi = [m.degrees(df['phi'].iloc[i]) for i in range(len(df))]
    return theta, phi


# define a function to convert cartesian coord. into spherical coordinates

def cart2sph(x, y, z):
    XsqPlusYsq = x ** 2 + y ** 2
    r = m.sqrt(XsqPlusYsq + z ** 2)  # r
    elev = m.atan2(z, m.sqrt(XsqPlusYsq))  # theta
    az = m.atan2(y, x)  # phi
    return [r, elev, az]


# define plot function

def plot_stereo_kmeans2(filepath, sph, size):
    theta = theta_phi(filepath, sph)[0]
    phi = theta_phi(filepath, sph)[1]
    fig = fig = plt.figure()
    # specify polar axes
    ax = fig.add_subplot(111, projection='equal_area_stereonet')
    for i in range(len(theta)):
        if theta[i] >= 0:
            ax.rake(phi[i] - 90, theta[i], -90, color='k', marker='.', markersize=12)
            ax.grid(linestyle='--')
            ax.set_azimuth_ticklabels([])
        else:
            theta[i] = np.pi - theta[i]
            ax.rake(phi[i] - 90, theta[i], -90, color='k', marker='.', markersize=12, mfc='white')
            ax.set_azimuth_ticklabels([])
    phi2 = [phi[i] - 90 + 0 * i for i in range(len(phi))]
    plot = ax.density_contourf(phi2, theta, -90, measurement='rakes', sigma=2, cmap='Reds')
    ax2 = fig.add_subplot(111, polar=True, facecolor="none")
    ax2.set_theta_direction(-1)  # reverse direction of degrees (clockwise)
    ax2.set_theta_zero_location('N')
    AzContours = range(0, 360, 45)
    AzLabels = [str(p) + '°' for p in AzContours]
    plt.thetagrids(AzContours, AzLabels)
    PlContours = [EqualArea(a) for a in range(30, 90, 30)]
    PlLabels = ["" for a in range(30, 90, 30)]
    plt.rgrids(PlContours, PlLabels)
    ax2.grid()
    cbaxes = fig.add_axes([0.85, 0.1, 0.03, 0.8])
    cb = fig.colorbar(plot, cax=cbaxes)
    plt.savefig('/Users/Jose/Desktop/Documents/Taenite/data/E00/plots/{}.png'.format(size), dpi=300)


# Equal-Area function
EqualArea = lambda Pl: np.sqrt(2.) * 90. * np.sin(np.radians(90. - Pl) / (2.))


# get the cartesian coordinates from *hyst files
def GetCart(filepath):
    cart = []
    for file in os.listdir(filepath)[:]:
        file_path = os.path.join(filepath, file)
        with open(file_path, 'r') as file:
            coord = file.readlines()[-1].split(',')
            mx = float(coord[-3])
            my = float(coord[-2])
            mz = float(coord[-1])
        cart.append([mx, my, mz])
    return cart


################################################
sizes = [10,12,14,16,18,20,22,24,26,28,30,35,40,45,50,60,70]
for size in sizes:
    # path to a directory containing lots of *.hyst files
    filepath = '/Users/Jose/Desktop/Documents/Taenite/data/E00/hysteresis/groundstates/{}/'.format(size)

    # Get the cartesian coordinates from these files
    cart = GetCart(filepath)

    # Convert the cartesian coord. into spherical coord. and
    # store it in the list sph
    sph = [cart2sph(i[0], i[1], i[2]) for i in cart]

    # plot
    plot_stereo_kmeans2(filepath, sph, size)